<a href="https://colab.research.google.com/github/KA-Jones/Voluntary_REDD_Analysis_GEE/blob/master/Create_Annual_Forest_Asset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Voluntary REDD Analysis (Google Earth Engine)

Generate annual time series measuring forest coverage in for all municipalities in the Brazilian Amazon. Annual land use/cover classifications from [MapBiomas](https://mapbiomas.org/) provide data between 1985 and 2019 using the Collection 5 dataset.



Developed by K.A. Jones, Department of Geography, San Diego State University


Based on Voluntary REDD Analysis developed by T. West.
[Github](https://github.com/thaleswest/Voluntary-REDD-analysis)

---


# Part 1. Create Annual Forest Asset


###**Input:** MapBiomas Collection 5

###**Output:** One image corrected for probited land cover transitions
The output image is saved as an Earth Engine asset which is analyzed in Part 2. Generate Forest Time Series. Once Part 1 is complete, the image asset may be used indefinitely. If a new MapBiomas Collection is released and/or the years included in the proposed study period change, Part 1 will need to be executed again to create a new asset that matches the project requirements.

**Please note**, due to how data are filtered and modified, changing the years of the study will influence the amaount of forest that is measured. Annual land cover is modified using temporal filters that "look" back and forward in time. Thus, changing the study years impacts what land cover data is available to process and generate annual forest cover time series.

---




To execute the code, you must have an account with Google Earth Engine. If you do not have an account, register and sign-up with Earth Engine at https://earthengine.google.com/.

### Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install).

***Uncomment these lines if you are running this notebook for the first time.***

In [ ]:
# !pip install earthengine-api

###Import Earth Engine (ee), Authenticate, and initialize Earth Engine API
You only need to authenticate the Earth Engine API once. Uncomment the line `ee.Authenticate()` 
if you are running this notebook for the first time or if you are getting an authentication error.  

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

### Define study years and import datasets.
Input: 

1.   Start and End Study Years (Collection 5: 1985-2019)
2.   MapBiomas `collection50_integration_v1`


[MapBiomas Collection 5 Classes](https://mapbiomas-br-site.s3.amazonaws.com/_PT-BR__C%C3%B3digos_da_legenda_Cole%C3%A7%C3%A3o_5.pdf)

In [ ]:
## Define Study Years (Start - End)
startDate = 1985
endDate = 2019

## Load data
mb_landcover = ee.Image('projects/mapbiomas-workspace/public/collection5/mapbiomas_collection50_integration_v1')

### Define Functions

In [ ]:
## Applied in Step 1
def reclass_img_to_col(yr):
  """Function 'reclass_img_to_col' creates one annual image for each study year using the original
  MapBiomas landcover image. Pixel values in each annual image are reclassified from the MapBiomas
  categories to new landcover categories: forest, non-forest, water, and cloud/NA."""
  year = ee.Number(yr)
  class_yr = ee.String('classification_').cat(year.format("%d"))
  img = mb_landcover.select(class_yr)
  img = img.remap(mapbiomas_lc, new_lc)
  img = img.rename(class_yr)                    # image bands are labeled 'remapped' following .remap(). Change back to 'classifcation_YYYY'
  img = img.set('YearID', year.format("%d"))    # Add new metadata property 'YearID' to be used in filtering collection
  img = img.set('date', year)                   # Add new metadata property 'date' to be used in filtering collection
  return img

## Applied in Step 2
def replace_cloud(current, previous):
  """Function 'replace_cloud' checks if pixel in current year is cloud/NA (0). If true, the
  current pixel value is updated to the pixel value of previous year."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(current.eq(0), last_image)
  return ee.List(previous).add(updated)

## Applied in Step 3
def mask_cloud(image):
  """Function 'mask_cloud' updates pixel values in all 'cloud_col' images
  based on cloud/NA (0) pixels in cloud_mask"""
  updated = image.where(cloud_mask.eq(1), 0)
  return updated

## Applied in Step 4
def change_water(current, previous):
  """Function 'change_water' checks if pixel in previous year is water (3) and same pixel
  in current year is not water. If true, the current pixel value is updated to water (3)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(3).And(current.neq(3)), 3)
  return ee.List(previous).add(updated)

## Applied in Step 5
def mask_water(image):
  """Function 'mask_water' is used to update pixel values in all 'water_col' images based on
  water (3) pixels in 'water_mask' making the water pixels consistent in all study years."""
  updated = image.where(water_mask.eq(1), 3)
  return updated

## Applied in Step 6
def to_nonforest(current, previous):
  """Funtion 'to_nonforest' flags forest to non-forest transition. If pixel is
  forest (1) in previous year and non-forest (2) in current year, flag current pixel (4)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(1).And(current.eq(2)), 4)
  return ee.List(previous).add(updated)

def repeat_nonforest(current, previous):
  """Funtion 'repeat_nonforest' checks if non-forest transition is stable at pixels in two sequential years.
  If pixel is flagged (4) in previous year and non-forest (2) in current year, flag current pixel (4)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(4).And(current.eq(2)), 4)
  return ee.List(previous).add(updated)

## Update 1 - correct pixels with Forest-> Non-Forest-> Non-Forest-> Forest transitions TO Forest-> Non-Forest-> Forest-> Forest
def return_nonfor_to_forest(image):
  """Function 'return_nonfor_to_forest' compares the current image to the next image. If the
  current image is flagged (4) and next image is forest (1), flagged pixels are changed back to
  forest."""
  img_date = ee.Number(image.get('date'))
  next = ee.Algorithms.If(img_date.lt(endDate),             # If image date is the final study year - no 'next' image.
                          ee.Image(flag_col.filterMetadata('date','equals', img_date.add(1)).first()),
                          image)
  updated = image.where(image.eq(4).And(ee.Image(next).eq(1)), 1)
  return updated

## Update 2 - correct pixels with Forest-> Non-Forest-> Forest-> Forest transitions TO Forest-> Forest-> Forest-> Forest
def repeat_return_nonfor_to_forest(image):
  """Function 'repeat_return_nonfor_to_forest' compares the current image to the next image. If the
  current image is flagged (4) and next image is forest (1), flagged pixels are changed back to
  forest. Relassify all remaining flagged pixels (.remap()) back to non-forest (2)"""
  img_date = ee.Number(image.get('date'))
  next = ee.Algorithms.If(img_date.lt(endDate),             # If image date is the final study year - no 'next' image.
                          ee.Image(fnff_updated_col.filterMetadata('date','equals', img_date.add(1)).first()),
                          image)
  updated = image.where(image.eq(4).And(ee.Image(next).eq(1)), 1)
  updated = updated.remap([0,1,2,3,4], [0,1,2,3,2])                             # Remap remaining flagged (4) pixels back to non-forest (2)
  updated = updated.rename(ee.String('Landcover_').cat(img_date.format("%d")))  # Rename to image band name from 'remapped' to 'Landcover'
  return updated

## Applied in Step 7
def fix_forest(current, previous):
  """Function 'fix_forest' fixes non-forest to forest transition. If pixel in previous year is non-forest (2)
  and same pixel in current year is forest (1), update current pixel back to non-forest (2)."""
  last_image = ee.Image(ee.List(previous).get(-1))
  current = ee.Image(current)
  updated = current.where(last_image.eq(2).And(current.eq(1)), 2)
  return ee.List(previous).add(updated)

### Reclassify the original MapBiomas land cover classes for years 1985 - 2019

In [ ]:
## Step 1. Reclassify the original MapBiomas land cover classes for years 1985 - 2019.
## Input: mb_landcover

## 1 = New/Reclassified forest - Based on original pixel values = 1, 2, 3, 4, and 5 (Do not include "forest plantation" [9] and "non forest natural formation" [10])
## 2 = New/Reclassified non-forest - Based on original pixel values = 9 to 15, 18 to 25, 29 to 32, 36, 39, and 41
## 3 = New/Reclassified water - Based on original pixel values = 26 and 33
## 0 = New/Reclassified cloud cover/NA - Based on the original "non observed" class (pixel value = 27)

## Lists of original MapBiomas landcover (lc) classes and NEW landcover classes
mapbiomas_lc = [1,2,3,4,5,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25,26,27,29,30,31,32,33,36,39,41]
new_lc = [1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,0,2,2,2,2,3,2,2,2]

## Create list of study years to map over
years_list = ee.List.sequence(startDate, endDate)

## Create image collection from band names
image_col = ee.ImageCollection.fromImages(ee.List(years_list.map(reclass_img_to_col)))


###------###
## Step 2. Replace Cloud/NA pixels (start year to last mapped year).
## Input: image_col from Step 1.

## Important: image_col is REVERSED so the process of updating pixels goes from endDate to startDate.
## If pixel in current image has a value of "0," get pixel value at same location in following year. The
## corrected image is then used in the next iteration (i.e. 2019 >> 2018, 2018 >> 2017, ...). After
## applying function 'replace_cloud', REVERSE the resulting list, 'cloud_col', back to the original order
## (i.e. startDate to endDate) for next steps.

## Reverse the input image collection by year (i.e. False = descending 2019 - 1985)
image_col_desc = image_col.sort('date', False)  

## Select first image in collection (last study year) as initial image to pass into iterate.
image_initial = ee.Image(image_col_desc.first())

## Use remaining images in collection (excluding the initial image) to iterate over.
images_to_iterate = image_col_desc.filterMetadata('date','less_than',endDate)

## Iterate over collection and apply 'replace_cloud' function. Output as new image collection, REVERSING
## the order of the image collection back to original (i.e. 1985 to 2019)
cloud_col = ee.ImageCollection(ee.List(images_to_iterate.iterate(replace_cloud, ee.List([image_initial])))).sort('date')


###------###
## Step 3. Apply cloud mask based on the second-to-last mapped year.
## Input: 'cloud_col' from Step 2 and a new layer, 'cloud_mask', created from the second-to-last 'cloud_col' image.

## Function 'mask_cloud' is used to update pixel values in all 'cloud_col' images based on cloud/NA (0) pixels in
## 'cloud_mask' to eliminate bias from cloud/NA pixels in final results. The updated images are output as
## cloud_mask_col for use in Step 5. If 2019 is the last mapped year, 2018 is used for cloud mask so the opportunity
## to replace NA/cloud (0) pixels in 2017 with the land use observed in 2019 (assuming those pixels are not cloud in 2019) exists.

## Create the cloud mask. Creates binary image (1 = cloud; 0 = no cloud).
cloud_mask = ee.Image(cloud_col.filterMetadata('date','equals', (endDate-1)).first()).eq(0)

## Apply function 'mask_cloud'
cloud_mask_col = cloud_col.map(mask_cloud)


###------###
## Step 4. Fix water transitions (1985 to last mapped year)
## Input: cloud_mask_col from Step 3

## Uses sequential annual classifications to identify temporally prohibited transitions. Results
## from the previous year are compared to the current year.

## Select first image in 'cloud_mask_col' collection as initial image to pass into iterate.
cloud_initial = ee.Image(cloud_mask_col.first())

## Use remaining images in 'cloud_mask_col' (excluding the initial image) to iterate over.
cloud_to_iterate = cloud_mask_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply function 'change_water'. Output as image collection.
water_col = ee.ImageCollection(ee.List(cloud_to_iterate.iterate(change_water, ee.List([cloud_initial]))))


###------###
## Step 5. Apply water mask based on the last mapped year.
## Input: water_col from Step 4 and a new layer, water_mask, created from the last water_col image.

## Function mask_water is used to update pixel values in all water_col images based on water (3) pixels
## in water_mask making the water pixels consistent in all study years. The results are added to water_mask_col for use in Step 7.

## Create the water mask. Creates binary image (1 = water; 0 = no water).
water_mask = ee.Image(water_col.filterMetadata('date','equals', endDate).first()).eq(3)

## Apply function 'mask_water'
water_mask_col = water_col.map(mask_water)


###------###
## Step 6. Identify and fix prohibited forest - non-forest - forest transitions.
## Input: water_mask_col from Step 5.

## Both functions use results from the previous image (backward comparison) and require .interate(). The first function
## flags (4) pixels undergoing the forest (1) to non-forest (2) transition. The second function checks to see if flagged (4)
## pixels remain non-forest (2) in the following year. If so, the non-forest (2) pixels are also flagged (4).
## 4 is the value used to flag pixels. F = Forest; N = Non-Forest

## Apply function 'to-nonforest'.
## Select first image in 'water_mask_col' collection as initial image to pass into iterate.
water_initial = ee.Image(water_mask_col.first())

## Use remaining images in 'water_mask_col' (excluding the initial image) to iterate over.
water_to_iterate = water_mask_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply change_water function. Output as image collection.
nonforest_col = ee.ImageCollection(ee.List(water_to_iterate.iterate(to_nonforest, ee.List([water_initial]))))


## Apply function 'repeat_nonforest'.
## Select first image in 'nonforest_col' collection as initial image to pass into iterate.
nonforest_initial = ee.Image(nonforest_col.first())

## Use remaining images in 'nonforest_col' (excluding the initial image) to iterate over.
nonforest_to_iterate = nonforest_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply 'repeat_nonforest' function. Output as image collection.
flag_col = ee.ImageCollection(ee.List(nonforest_to_iterate.iterate(repeat_nonforest, ee.List([nonforest_initial]))))

###----###
## With "forest - nonforest" and "forest - nonforest - nonforest" pixels flagged, compare the current image to
## the next image (forward comparison). If pixels are flagged (4) in Image n and forest (1) in Image n+1, update the
## flagged pixels Image n to forest (1). The process to update pixel values is repeated twice to fix consecutive years
## of flagged pixels (i.e. F-N-N-F to F-N-F-F and then F-N-F-F to F-F-F-F):

## 1) forest to non-forest (flagged) to forest
## 2) forest to non-forest (flagged) to non-forest (flagged) to forest

## Update 1) map the return_nonfor_to_forest function over flag_col image collection.

## Update 2) map the repeate_return_nonfor_to_forest function over fnff_updated_colimage collection. After the final
## updates, relassify all remaining flagged pixels (.remap()) back to non-forest (2).


## Update 1 - correct from F-N-N-F to F-N-F-F
## Apply function 'return_nonfor_to_forest'
fnff_updated_col = flag_col.map(return_nonfor_to_forest)

## Update 2 - correct from F-N-F-F to F-F-F-F
## Apply function 'repeat_return_nonfor_to_forest'
ffff_updated_col = fnff_updated_col.map(repeat_return_nonfor_to_forest)


###------###
## Step 7. Fix logical errors in non-forest - forest transition.
## Input: ffff_updated_col from Step 6.

## The fix_forest function identifies non-forest (2) to forest (1) transition pixels. It changes forest (1)
## pixels back to non-forest (2) to avoid false reforestation that will result in overestimation of forest.

## Apply function 'fix_forest'.
## Select first image in 'ffff_col' collection as initial image to pass into iterate.
ffff_initial = ee.Image(ffff_updated_col.first())

## Use remaining images in 'ffff_col' (excluding the initial image) to iterate over.
ffff_to_iterate = ffff_updated_col.filterMetadata('date','greater_than', startDate)

## Iterate over collection and apply fix_forest function. Output as final image collection.
final_collection = ee.ImageCollection(ee.List(ffff_to_iterate.iterate(fix_forest, ee.List([ffff_initial]))))    # FINAL IMAGE COLLECTION

## Convert image collection into final image.
final_image = final_collection.toBands()

## Apply regular expresion to modify band names by eliminating number and underscore prefix (i.e. 1_Landcover_1985 to Landcover_1985)
final_image = final_image.regexpRename('\d+_(.*)', '$1')

### Export Corrected Land Cover Image

Set `description` to define name of task.

Set `assetId` to define Google Earth Engine project folder (i.e. `Biggs-Lab`) and the asset **name** (i.e. `REDD_MapBiomas_Col5_85_19`).

The `assetId` **name** is what will appear in your personal [Earth Engine Code Editor](https://code.earthengine.google.com/) **Assets** folder.



---

###Use the new image asset to generate forest cover time series.
The approximate export time is between 4-6 hours.

In [ ]:
## Define the task in Python
task = ee.batch.Export.image.toAsset(final_image, description= 'V_REDD_to_Asset_85_19', assetId= 'projects/Biggs-Lab/REDD_MapBiomas_Col5_85_19', maxPixels=1e13)

## Start the task, equivalent to hit the "run" button in the editor
task.start()